# Eat Safe, Love


## Notebook Set Up

In [20]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import json

# DO NOT FORGET TO RUN THIS COMMAND WIHTIN THE TERMINAL TO IMPORT THE JSON FILE 
# mongoimport --type json -d uk_food -c establishments --drop --jsonArray "file:///C:/Users/Leo/OneDrive/Desktop/establishments.json"


In [21]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [22]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [23]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [24]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [25]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene' : 20 }
results = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
print('The number of establishemnts with a hygiene score of 20:',establishments.count_documents(query))
# Display the first document in the results using pprint
print('The first establishment with the hygiene score of 20 is:')
print('  ')
pprint(establishments.find_one(query))

The number of establishemnts with a hygiene score of 20: 0
The first establishment with the hygiene score of 20 is:
  
None


In [26]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print('number of rows:', results_df.shape[0])
# Display the first 10 rows of the DataFrame
results_df.head(10)

number of rows: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [27]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_2 = {'LocalAuthorityName': {'$regex': 'London'},
    'RatingValue': {'$gte': 4}
}

# Use count_documents to display the number of documents in the result
rating_count = establishments.count_documents(query_2)
print('The number of establishments in London with a RatingValue >= 4:', rating_count)

# Display the first document in the results using pprint
print('The first establishment with the RatingValue >= 4:')
print('  ')
pprint(establishments.find_one(query_2))

The number of establishments in London with a RatingValue >= 4: 0
The first establishment with the RatingValue >= 4:
  
None


In [28]:
# Convert the result to a Pandas DataFrame
Rating_results_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print('number of rows:', Rating_results_df.shape[0])

# Display the first 10 rows of the DataFrame
Rating_results_df.head(10)

number of rows: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [29]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.083812
longitude = 1.195625

query_3 = {'RatingValue': '5', 
         "geocode.longitude": {'$gte': (longitude - degree_search), '$lte': (longitude + degree_search)}, 
         "geocode.latitude": {'$gte': (latitude - degree_search), '$lte': (latitude + degree_search)}}
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))

for result in results:
    pprint(result)


In [30]:
# Convert result to Pandas DataFrame
top_5_results_df = pd.DataFrame(results)
top_5_results_df.head(5)

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [31]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene' : 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group' : {'_id': '$LocalAuthorityName',
                           'count' : {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sort_query  = {'$sort': {'count': -1 }}

#Create the pipeline
pipeline = [match_query, group_query, sort_query]

#results
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print('Number of documents in the results:' , len(results))
# Print the first 10 results
pprint(results[0:10])

Number of documents in the results: 0
[]


In [32]:
# Convert the result to a Pandas DataFrame
Low_score_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print('number of rows:', Low_score_df.shape[0])
# Display the first 10 rows of the DataFrame
Low_score_df.head(10)

number of rows: 0


""
